Here we will build prediction model based on the data produced at the ETL stage. We will compare:
- baseline, which will be a linear model based on 1 feature "number_of_panels_total"
- linear model with all the rest features
- linear model with features produced by one layer MLP
- random forest
- adaboost
- xgboost

In [22]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Data initiation

In [4]:
df = pd.read_csv('datasets/modeling_df.csv')
all_feats = [
    'lat_max',
    'lat_min',
    'long_max',
    'long_min',
    'number_of_panels_north',
    'number_of_panels_south',
    'number_of_panels_east',
    'number_of_panels_west',
    'number_of_panels_flat',
    'number_of_panels_total',
    'lat_mean',
    'long_mean',
    'zip_area',
    'dens',
    'approx_zips',
    'is_usa',
    'zip_lat',
    'zip_long',
    'state_size',
    'state_lat',
    'state_long'
]
label = 'potential_kwh_total'

# Baseline

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df[['number_of_panels_total']].values, df[[label]].values, test_size=.2)
baseline_model = linear_model.LinearRegression(positive=True)
y_pred = baseline_model.fit(X_train, y_train).predict(X_test)


In [33]:
baseline_rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_test))
baseline_rmse

9.991230584437321

In [35]:
np.std(df[label]) / baseline_rmse

6.62191730878738

In [31]:
iqr = df[label].quantile(.75) - df[label].quantile(.25)
iqr / baseline_rmse

8.765223676391809

In [28]:
r2_score(y_pred=y_pred, y_true=y_test)

0.9763339600838112

Baseline model is only about 17% of label std and about 12% of iqr and very high r2 score. It will probably be hard to bit.